In [1]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [5]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [6]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [7]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\jshac\AppData\Local\Temp/ipykernel_16044/1455435457.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


In [8]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [9]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x


net = Net(dic_size, hidden_size, 2)

In [10]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
optimizer.zero_grad()
outputs = net(X)
loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
loss.backward()
optimizer.step()

print(outputs.view(-1, dic_size), outputs.view(-1, dic_size).shape)
results = outputs.argmax(dim=2)
print(results, results.shape)

predict_str = ""
for j, result in enumerate(results):
    # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
    if j == 0:
        predict_str += ''.join([char_set[t] for t in result])
    else:
        predict_str += char_set[result[-1]]

print(predict_str)

tensor([[-10.0788,  -3.8773,  -1.8639,  ...,  -4.3506,  -6.6645,   1.0475],
        [  3.0957,  -3.7871,  -7.6300,  ...,   1.5969,   6.2869,   2.1575],
        [ 13.7043,  -8.0602,   7.9613,  ...,   2.9496,   4.8526,   2.6435],
        ...,
        [ -1.8999,  13.9442,  -3.1699,  ...,  -3.2876,   7.1303,   1.0955],
        [ -1.7762,   4.3324,  -5.6526,  ...,   5.6114,  13.6354,   4.2337],
        [ -8.9214,  -4.0760,  -2.3681,  ...,   1.7034,  -2.8357,   1.3780]],
       grad_fn=<ViewBackward0>) torch.Size([1700, 25])
tensor([[ 4, 17,  0,  ..., 23,  5,  7],
        [15,  0, 15,  ...,  5,  7, 17],
        [ 7, 15, 13,  ...,  7, 17,  7],
        ...,
        [17, 15, 14,  ..., 17, 22,  1],
        [ 7, 14, 17,  ..., 22,  1, 23],
        [17, 17,  7,  ...,  1, 23, 20]]) torch.Size([170, 10])
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [13]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    print(outputs.view(-1, dic_size), outputs.view(-1, dic_size).shape)
    results = outputs.argmax(dim=2)
    print(results, results.shape)

    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

tensor([[-10.1325,  -2.2570,  -2.5151,  ...,  -4.6934,  -5.9437,   1.0342],
        [  1.9867,  -2.5990,  -7.2139,  ...,   1.7822,   5.2649,   0.8318],
        [ 12.3633,  -7.9776,   6.9399,  ...,   2.9820,   2.7263,   2.7073],
        ...,
        [ -0.8183,  10.4132,  -3.8707,  ...,  -4.5164,   4.9509,   3.1259],
        [ -3.6613,   4.3224,  -6.1175,  ...,   7.6371,  13.6847,   3.3875],
        [-10.6655,  -1.4755,  -5.0856,  ...,   4.4304,  -4.0509,  -3.1651]],
       grad_fn=<ViewBackward0>) torch.Size([1700, 25])
tensor([[ 6, 17,  0,  ..., 23,  5,  7],
        [15,  0, 15,  ...,  5,  7, 17],
        [ 7, 15, 13,  ...,  7, 17,  7],
        ...,
        [15, 15, 14,  ..., 17, 22,  1],
        [ 7, 14, 17,  ..., 22,  1, 23],
        [17, 17,  7,  ...,  1, 23, 20]]) torch.Size([170, 10])
g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
tensor([[-10.0884, 